# Sundial problem setup

This notebook contains some setup for the sundial problem in [SymPy](https://www.sympy.org/en/index.html) and [GAlgebra](https://github.com/pygae/galgebra).

In [1]:
import sympy
from sympy import sin, cos, tan
from sympy.abc import *
from galgebra import metric, mv
from galgebra.ga import Ga
from galgebra.printer import latex
from IPython.display import Math

# tell sympy to use galgebra printing
sympy.init_printing(
    latex_printer=latex,
    use_latex='mathjax',
    omit_function_args=True,
    omit_partial_derivative_fraction=True,
    )

First, we define the geometric algebra of 3-space and some basis blades from the frame of the "fixed stars"

![Earth's orientation and orbit](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/MainArena.svg?token=GHSAT0AAAAAAB73Q3JM6JGFDMRHPPAJKRQ2ZAWFO7Q "Earth's orientation and orbit.").

In [2]:
coords = sympy.symbols('1 2 3', real=True)
G3 = Ga('e', g=[1,1,1], coords=coords)

(e1, e2, e3) = G3.mv()
I = e1^e2^e3

Now, let the tilt of the earth's plane (axis) of rotation be $\alpha$ and measure the earth's rotation by $\psi$. Then we can define the Earth frame as follows.

In [3]:
R_alpha = cos(alpha/2) - (e1^e3) * sin(alpha/2)

def rotate(mv, rotor):
    return ( rotor * mv * rotor.rev() )

e1_prime = rotate(e1, R_alpha).trigsimp()
R_psi = (cos(psi/2) - (e1_prime^e2 * sin(psi/2))).trigsimp()

f1 = rotate(e1_prime, R_psi).trigsimp().trigsimp()
f2 = rotate(e2, R_psi).trigsimp().trigsimp()
f3 = rotate(e3, R_alpha).trigsimp().trigsimp()

def print_3frame(frame, symbol):
    return Math(fr'''
            \begin{{align}}
            {symbol}_1 &= {latex(frame[0])} \nonumber \\
            {symbol}_2 &= {latex(frame[1])} \nonumber \\
            {symbol}_3 &= {latex(frame[2])} \nonumber
            \end{{align}}
            ''')

print_3frame((f1,f2,f3), "f")

<IPython.core.display.Math object>

Check that this is an orthonormal frame

In [4]:
result = (e1 ^ e2 ^ e3) - (f1 * f2 * f3)
assert result.obj.equals(0)

The equatorial plane should only depend on the tilt of the Earth's axis of spin $\alpha$, not the angle by which it has rotated relative to the fixed stars $\psi$.

In [5]:
f1^f2

cos(alpha)*e_1^e_2 - sin(alpha)*e_2^e_3

Define an orthonormal frame embedded in the Earth's surface, with $n_1$ pointing South, $n_2$ pointing East and $n_3$ pointing up.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.svg?token=GHSAT0AAAAAAB73Q3JNXNOAJWYLCUINTVLUZAWF6JQ "Frame embedded in Earth's surface.").

In [6]:
R_theta = cos(theta/2) - ( (f3^f1) * sin(theta/2) )

n1 = rotate(f1, R_theta).trigsimp().trigsimp()
n2 = f2

# n3 needs a little love
raw_n3 = rotate(f3, R_theta).obj.trigsimp()
sympy_n3 = sympy.expand(sympy.expand_trig(raw_n3)) # galgebra's Mv doesn't have expand_trig as a method
n3 = mv.Mv(sympy_n3, ga=G3)

print_3frame((n1,n2,n3), "n")

<IPython.core.display.Math object>

Earth orbit rotor $R_\sigma$, and vector parallel to rays of sunshine, $s$.

In [7]:
R_sigma = cos(sigma/2) - (e1^e2)*sin(sigma/2)
s = rotate(e1, R_sigma).trigsimp()

def print_eq(lhs : str, rhs : sympy.Symbol):
    return Math(fr'''
        \begin{{equation}}
            {lhs} = {latex(rhs)} \nonumber
        \end{{equation}}
        ''')

print_eq("s", s)

<IPython.core.display.Math object>

Dial face and gnomon. Define an orthnormal frame $u_1, u_2, u_3$ as the unevaluated version of $n_1, n_2, n_3$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/DialFrame.svg?token=GHSAT0AAAAAAB73Q3JNJN46TIEHP3QCWWGYZAWGADA "Frame embedded in the sundial's face.").

In [8]:
G3n = Ga('u', g=[1,1,1], coords=coords)

(u1, u2, u3) = G3n.mv()
U = u1^u2^u3

Dial face expressed relative to $u$ basis: $G_u$.

In [9]:
R_i = cos(i/2) - (u1^u3)*sin(i/2)
R_d = cos(d/2) - (u1^u2)*sin(d/2)

Gu = rotate(rotate(u1^u2, R_i), R_d).trigsimp()
print_eq("G_u", Gu)

<IPython.core.display.Math object>

Frame embedded in dial face.

In [10]:
m1 = rotate(rotate(u1, R_i), R_d).trigsimp()
m2 = rotate(rotate(u2, R_i), R_d)
m3 = rotate(rotate(u3, R_i), R_d).trigsimp()

print_3frame((m1,m2,m3), "m")

<IPython.core.display.Math object>

The gnomon expressed relative to the $u$ frame, $g_u$.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/Gnomon.svg?token=GHSAT0AAAAAAB73Q3JNHJMDP6T55SWPTQFGZAWGBNA "The gnomon.").


In [11]:
R_iota = cos(iota/2) - (u1^u3)*sin(iota/2)
R_delta = cos(delta/2) - (u1^u2)*sin(delta/2)

gu = rotate(rotate(u3, R_iota), R_delta).trigsimp()
print_eq("g_u", gu)

<IPython.core.display.Math object>

The meridian plane, $M$.

In [12]:
M = (n1^n3).trigsimp()
print_eq("M", M)

<IPython.core.display.Math object>

The noon line is the intersection of the sunshine vector $s$ and the meridian plane $M$, which occurs where $s \wedge M$ vanishes.

In [13]:
(s^M).trigsimp()

(sin(psi)*cos(alpha)*cos(sigma) - sin(sigma)*cos(psi))*e_1^e_2^e_3

In [14]:
coeff = _.trigsimp().get_coefs(3)[0]
soln = sympy.solve(coeff.subs(sin(psi), tan(psi)*cos(psi)), tan(psi))[0]
print_eq( r"\tan(\psi)", soln )

<IPython.core.display.Math object>

Gnomon lies in meridian plane when its declination angle $\delta$ is zero.

In [15]:
gu = gu.subs(delta, 0)
print_eq("g_u", gu)

<IPython.core.display.Math object>

$S = s \wedge g$ is the plane containing the sunshine vector and the gnomon.

In [16]:
def _angle_replacements(angles):
    # make eg {'s_alpha':sin(alpha), 'c_alpha':cos(alpha)}
    replacements = {}
    for angle in angles:
        replacements[sympy.symbols('s_' + str(angle))] = sin(angle)
        replacements[sympy.symbols('c_' + str(angle))] = cos(angle)
    return replacements

def hide_angles(expr, angles):
    for hidden_expr, trig_expr in _angle_replacements(angles).items():
        expr = expr.subs(trig_expr, hidden_expr)
    return expr

def unhide_angles(expr, angles):
    for hidden_expr, trig_expr in _angle_replacements(angles).items():
        expr = expr.subs(hidden_expr, trig_expr)
    return expr

In [17]:
nh = [hide_angles(ni, [alpha, psi, theta]) for ni in (n1, n2, n3)]
guh = hide_angles(gu.trigsimp(), [iota])
gh = sum([ c*ni for c, ni in zip(guh.get_coefs(1), nh)])
sh = hide_angles(s, [sigma])
S = unhide_angles(sh^gh, [alpha, psi, theta, iota, sigma])
S.Fmt(3)

 (sin(alpha)*sin(sigma)*cos(iota - theta) - sin(psi)*sin(iota - theta)*cos(sigma) + sin(sigma)*sin(iota - theta)*cos(alpha)*cos(psi))*e_1^e_2
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*cos(sigma)*e_1^e_3
 + (-sin(alpha)*sin(iota - theta)*cos(psi) + cos(alpha)*cos(iota - theta))*sin(sigma)*e_2^e_3

Define the solar hour angle $\mu$ as that between $S$ and the meridian plane $M$.

In [18]:
S|M

sin(alpha)*sin(iota - theta)*cos(sigma) - sin(psi)*sin(sigma)*cos(iota - theta) - cos(alpha)*cos(psi)*cos(sigma)*cos(iota - theta)

In [19]:
c_mu = sympy.collect(sympy.trigsimp(_.obj), cos(iota-theta))
print_eq(r"\cos\mu", c_mu)

<IPython.core.display.Math object>